In [1]:
using Revise, NeuralNetworkAnalysis, NeuralVerification, YAML
const NV = NeuralVerification
const RA = ReachabilityAnalysis
const IA = IntervalArithmetic
const NNA = NeuralNetworkAnalysis

┌ Info: Precompiling NeuralNetworkAnalysis [73c0b437-a350-4e9b-97ac-9adb151c271b]
└ @ Base loading.jl:1278
┌ Warning: Replacing docs for `LazySets.HPolytope :: Union{Tuple{Union{Polyhedra.HRepresentation{N}, Polyhedra.Polyhedron{N}}}, Tuple{N}} where N` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `LazySets.HPolyhedron :: Union{Tuple{Union{Polyhedra.HRepresentation{N}, Polyhedra.Polyhedron{N}}}, Tuple{N}} where N` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `Polyhedra.polyhedron :: Union{Tuple{Union{HPolyhedron{N,VN} where VN<:AbstractArray{N,1}, HPolytope{N,VN} where VN<:AbstractArray{N,1}}}, Tuple{N}} where N<:Real` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227
┌ Warning: Replacing docs for `Polyhedra.polyhedron :: Union{Tuple{VPolytope{N,VN} where VN<:AbstractArray{N,1}}, Tuple{N}} where N<:Real` in module `LazySets`
└ @ Base.Docs docs/Docs.jl:227


NeuralNetworkAnalysis

In [2]:
d = YAML.load_file("sig16x16.yml")
nnet = NNA.read_yaml(d);

In [46]:
X0 = Hyperrectangle(low=[-0.6, 0], high=[-0.4, 0])
X0s = split(X0, [1, 1])
sol = [NNA.forward(nnet, X0) for X0 in X0s];

In [43]:
overapproximate(UnionSetArray(sol), Interval) # 8

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.452969, -0.0564312])

In [45]:
overapproximate(UnionSetArray(sol), Interval) # 4

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.467717, -0.0640337])

In [47]:
overapproximate(UnionSetArray(sol), Interval) # 1

Interval{Float64,IntervalArithmetic.Interval{Float64}}([-0.448458, -0.0581502])

In [52]:
function step(p, v, u)
    return (p + v, v + 0.0015u - 0.0025 * cos(3p))
end

step (generic function with 1 method)

In [54]:
p, v = -0.4..0.6, 0. .. 0.

([-0.400001, 0.600001], [0, 0])

In [61]:
function mountain_car(p, v, nnet; max_setps=150)
    reward = 100.
    n = 0
    for i=1:max_setps
        n += 1
        u = forward(nnet, Interval(p) × Interval(v))
        u = u.array[1].dat
        reward = reward - 0.1 * u^2
        p, v = step(p, v, u)
        if p.lo >= 0.45
            break
        end
    end
    return p, v, reward, n
end

mountain_car (generic function with 1 method)

In [62]:
@time mountain_car(p, v, nnet)

LoadError: KeyError: key :xv not found

In [23]:
@taylorize function car!(dx, x, p, t)
    p, v, u = x
    dx[1] = p + v
    dx[2] = v + 0.0015u - 0.0025 * cos(3p)
    dx[3] = zero(u)

    return dx
end

X₀ = Hyperrectangle(low=[-0.41, 0], high=[-0.4, 0])
U₀ = Universe(1)
prob = @ivp(x' = car!(x), dim: 3, x(0) ∈ X₀ × U₀)
vars_idx = Dict(:state_vars=>1:2, :input_vars=>[], :control_vars=>[3]);

In [27]:
plant = ControlledPlant(prob, nnet, vars_idx);
alg = TMJets(abs_tol=1e-9, orderT=8, orderQ=1)
solver = "hybrid"

@time sol = NNA.solve(plant, T=0.6, Tsample=0.2, alg_nn=solver, alg=alg)
#solz = overapproximate(sol, Zonotope);

  6.679532 seconds (76.88 M allocations: 6.025 GiB, 28.19% gc time)


In [28]:
sol

BoundsError: BoundsError: attempt to access String
  at index [2]